### 테스트용 이미지 다운로드

In [ ]:
!wget https://www.sciencenews.org/wp-content/uploads/2020/03/033120_HT_covid-cat_feat-1028x579.jpg
!ls -lia

### PIL로 시각화 및 torch transforms 적용해 보기

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

org_img_pil = Image.open('033120_HT_covid-cat_feat-1028x579.jpg')

def show_image(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    
show_image(org_img_pil)
print(type(org_img_pil))


In [ ]:
from torchvision import transforms as T

# 좌우 반전을 위한 transforms RandomHorizontalFlip 객체 생성. 
transform = T.RandomHorizontalFlip(p=1)

# 좌우 반전 수행. 입력인자로 PIL 객체 또는 Tensor객체를 입력 받음.
# 입력 PIL 시 좌우 반전 수행 완료된 PIL을 반환
aug_img = transform(org_img_pil)

print(type(aug_img))
show_image(aug_img)

### opencv로 읽어들인 Numpy array는 transforms 에 적용할 수 없음
* opencv의 imread(파일명)은 이미지 파일을 numpy 형태로 읽어들임. (높이, 너비, 채널수) 형태의 shape을 가지며 Channel Last임
* imread()로 읽어들인 Channel 순서는 RGB가 아니라 BGR임. 이를 RGB로 변경하기 위해 cvtColor(array, cv2.COLOR_BGR2RGB)

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

org_img_numpy = cv2.cvtColor(cv2.imread('033120_HT_covid-cat_feat-1028x579.jpg'), cv2.COLOR_BGR2RGB)

def show_image(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    
show_image(org_img_numpy)
print(type(org_img_numpy), org_img_numpy.shape)


In [ ]:
from torchvision import transforms as T
import cv2

# opencv 는 image를 (h, w, c) 즉 channel last로 읽어 들임. 그리고 값은 unsigned int 형임.
org_img_numpy = cv2.cvtColor(cv2.imread('033120_HT_covid-cat_feat-1028x579.jpg'), cv2.COLOR_BGR2RGB)

print(type(org_img_numpy), org_img_numpy.dtype)
# 좌우 반전 객체 생성. 
transform = T.RandomHorizontalFlip(p=1)

# transforms는 PIL 또는 pytorch tensor가 입력 되어야 만 함.
aug_img = transform(org_img_numpy)

print(type(aug_img))
show_image(aug_img)


### Numpy array를 torchvision transforms에 부득이하게 사용해야 할 경우
* numpy array를 PIL이나 Tensor로 변환
* Tensor변환 시 Channel Last numpy array가 Channel First Tensor로 변환이 되어야 함. 

In [ ]:
'''
Numpy array를 PIL로 변환 후 transforms 적용
'''
# opencv 는 image를 (h, w, c) 즉 channel last로 읽어 들임. 그리고 값은 unsigned int 형임.
org_img_numpy = cv2.cvtColor(cv2.imread('033120_HT_covid-cat_feat-1028x579.jpg'), cv2.COLOR_BGR2RGB)
# numpy array를 PIL 객체로 변환
org_img_pil = Image.fromarray(org_img_numpy)

# 좌우 반전 객체 생성. 
transform = T.RandomHorizontalFlip(p=1)
aug_img = transform(org_img_pil)

show_image(aug_img)


In [ ]:
'''
Numpy array를 Tensor로 변환 후 Transforms 적용. Channel Last -> Channel First 변경 및 float 32로 dtype 변경 
'''
import torch

org_img_numpy = cv2.cvtColor(cv2.imread('033120_HT_covid-cat_feat-1028x579.jpg'), cv2.COLOR_BGR2RGB)
# numpy를 tensor로 변환하되, channel first로 변환하고 uint8값을 float32로 변경해 줌. 
org_img_tensor = torch.tensor(org_img_numpy).permute(2, 0, 1).to(torch.float32)
print(type(org_img_tensor), org_img_tensor.dtype, org_img_tensor.shape)

# 좌우 반전 객체 생성. 
transform = T.RandomHorizontalFlip(p=1)
aug_img_tensor = transform(org_img_tensor)
print(type(aug_img_tensor), aug_img_tensor.dtype, aug_img_tensor.shape)

# 시각화를 위해서 PIL로 다시 변환. ToPILImage() 적용시 tensor는 channel first이되 dtype은 uint8로 변경되어야 함. 
to_pil = T.ToPILImage()
aug_pil_image = to_pil(aug_img_tensor.to(torch.uint8))

print(type(aug_pil_image))
# show_image(aug_img)
plt.imshow(aug_pil_image)

In [ ]:
org_img_tensor

### Augmentation 적용 확률 적용 및 RandomHorizontalFlip과 RandomVerticalFlip
* p 인자를 통해 해당 augmentation을 적용할지, 원본 image를 그대로 유지할 지의 확률 설정. 
* RandomHorizontalFlip은 좌우 반전, RandomVerticalFlip은 상하 반전

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms as T
#from torchvision.transforms import v2

org_img_pil = Image.open('033120_HT_covid-cat_feat-1028x579.jpg')
# 좌우 반전 적용
transforms_h = T.RandomHorizontalFlip(p=0.5)
aug_img = transforms_h(org_img_pil)

show_image(aug_img)

In [ ]:
# 상하 반전 적용
transform_v = T.RandomVerticalFlip(p=0.5)
aug_img = transform_v(org_img_pil)
show_image(aug_img)

In [ ]:
# 여러개의 이미지를 subplots로 시각화
def show_images(images, labels, ncols=5, title=None):
    figure, axs = plt.subplots(figsize=(28, 6), nrows=1, ncols=ncols)
    for i in range(ncols):
        axs[i].imshow(images[i])
        axs[i].set_title(labels[i])

# augmentor로 입력된 albumentations augmentation을 반복 수행
def repeat_aug(count=4, org_image=None, label=None, transform=None):
    image_list = [org_image]
    label_list = ['original']

    for i in range(count):
        aug_image = transform(org_image)
        image_list.append(aug_image)
        label_list.append(label)
        
    show_images(image_list, label_list, ncols=count+1)
    
    
transform = T.RandomHorizontalFlip(p=0.5)
repeat_aug(count=4, org_image=org_img_pil, label='HorizontalFlip', transform=transform)

In [ ]:
# 상하 반전 연속 적용
transform = T.RandomVerticalFlip(p=0.5)

repeat_aug(count=4, org_image=org_img_pil, label='VerticalFlip', transform=transform)

### Compose() 를 이용하여 여러 Augmentation를 체인 형태로 연속 적용.
* 인자로 연속으로 적용할 transforms 객체를 List로 입력함.

In [ ]:
transform = T.Compose([
    T.RandomVerticalFlip(p=0.5),
    T.RandomHorizontalFlip(p=0.5)
    ])

repeat_aug(count=6, org_image=org_img_pil, label='Composite', transform=transform)
repeat_aug(count=6, org_image=org_img_pil, label='Composite', transform=transform)

### CenterCrop, RandomCrop, RandomReSizedCrop
* Crop은 입력된 height, width 값 만큼의 원본 이미지의 특정 영역을 잘라낸 후 원본 사이즈로 다시 Resize하지 않음.
* CenterCrop은 원본 이미지의 중심을 기준으로 입력된 height, width 값 만큼의 영역을 Crop 함(원본 이미지 사이즈로 Resize하지 않음)
* RandomCrop은 입력된 height, width 값 만큼의 영역을 Random하게 Crop.(원본 이미지 사이즈로 Resize하지 않음)
* RandomResizedCrop은 원본 이미지의 Random한 영역을 잘라낸 후, 입력된 height, width값의 사이즈로 Resize함
* RandomResizedCrop은 어떤 영역을 얼마만큼 잘라내는 지는 알수 없으며, 대략적으로 scale과 ratio를 조정하여 적용.
* CenterCrop, RandomCrop, RandomResizedCrop 모두 Probability 인자를 가지지 않음. 이의 적용을 위하여 RandomApply 사용. 
* CenterCrop, RadomCrop은 적용 후 pytorch Model에 입력될 이미지 사이즈로 resize 적용 해야 함. 

In [ ]:
# CenterCrop은 원본 이미지의 중심을 기반으로 size 만큼을 추출함.
# probability를 인자로 가지고 있지 않음. RandomApply로 감싸서 호출이 필요. 
transform = T.CenterCrop(size=(224, 224)) #또는 size=224
repeat_aug(count=6, org_image=org_img_pil, label='CenterCrop', transform=transform)

# probability를 인자로 가지고 있지 않음. RandomApply로 감싸서 호출이 필요. RandomApply()의 인자로 List필요하며 default p는 0.5임.
transform = T.RandomApply([T.CenterCrop(size=(224, 224))], p=0.5)
repeat_aug(count=6, org_image=org_img_pil, label='CenterCrop', transform=transform)

#최종 이미지가 만들어지므로 Resize를 이용하여 다른 이미지 크기로 재 변환이 필요할 수 있음.
transform = T.RandomApply([T.CenterCrop(size=(224, 224)), T.Resize(size=(448, 448))], p=0.5)
repeat_aug(count=6, org_image=org_img_pil, label='CenterCrop', transform=transform)


In [ ]:
# Compose 체인에 CenterCrop, RandomCrop, Crop등의 Crop계열 변환이 사용되었을 경우 고정된 이미지 크기를 위해서 맨 마지막에 Resize변환 필요
transform = T.Compose([
    T.RandomVerticalFlip(p=0.5),
    T.RandomApply([T.CenterCrop(size=(224, 224))], p=0.5),
    T.Resize(size=(448, 448))
    ])

repeat_aug(count=6, org_image=org_img_pil, label='CenterCrop_Comp', transform=transform)

In [ ]:
# RandomCrop은 입력된 height, width 값 만큼의 영역을 Random하게 Crop.(원본 이미지 사이즈로 Resize하지 않음)
transform = T.RandomCrop(size=(224, 224))
repeat_aug(count=6, org_image=org_img_pil, label='RandomCrop', transform=transform)

transform = T.RandomCrop(size=(1280, 1280), pad_if_needed=True) #  padding_mode는 'constant', 'edge', 'reflect', 'symmetric'
repeat_aug(count=6, org_image=org_img_pil, label='RandomCrop', transform=transform)

transform = T.Compose([
    T.RandomVerticalFlip(p=0.5),
    T.RandomApply([T.RandomCrop(size=(224, 224))], p=0.5),
    T.Resize(size=(448, 448))
    ])

repeat_aug(count=6, org_image=org_img_pil, label='RandomCrop_Comp', transform=transform)

In [ ]:
# RandomResizedCrop은 원본 이미지의 Random한 영역을 잘라낸 후, 입력된 height, width값의 사이즈로 Resize함
# RandomResizedCrop은 어떤 영역을 얼마만큼 잘라내는 지는 알수 없으며, 대략적으로 scale과 ratio를 조정하여 적용.
# scale은 전체 image에서 잘라낼 비율영역(최소, 최대), ratio는 상하 비율 0.75=3:4(taller), 1.333=4:3(wider)
transform = T.RandomResizedCrop(size=(224, 224)) # default scale은 (0.08, 1.0), ratio는 (0.75, 1.333)
repeat_aug(count=6, org_image=org_img_pil, label='RandomResizedCrop', transform=transform)

# scale인자를 적용하여 crop되는 위치 변동성을 줄일 수 있음.
transform = T.RandomResizedCrop(size=(224, 224), scale=(0.3, 0.8))
repeat_aug(count=6, org_image=org_img_pil, label='RandomResizedCrop', transform=transform)

### RandomRotation
* degrees에 정해진 범위내에서 회전. degrees=90 또는 (-90, 90) 이면 -90~90도 사이에 random 회전.
* p 인자가 없으므로 RandomApply로 적용 해야 함. 


In [ ]:
transform = T.RandomRotation(degrees=90) # degrees=(-90, 90), expand=True이면 rotate 적용된 이미지가 모두 포함되도록 이미지 크기를 늘림
repeat_aug(count=6, org_image=org_img_pil, label='Rotate', transform=transform)

transform = T.RandomApply([T.RandomRotation(degrees=90)], p=0.5)
repeat_aug(count=6, org_image=org_img_pil, label='Rotate', transform=transform)

### RandomAffine
* Rotation(회전), Shift(상하/좌우 이동), Scale(이미지 확장)을 한꺼번에 적용. 개별 적용은 Random 적용
* p인자가 없으므로 RandomApply를 감싸서 적용. 

In [ ]:
transform = T.RandomAffine(degrees=30, # -30 ~ 30도 Random 회전
                            translate=(0.1, 0.1), # horizontal과 vertical방향으로 Random shift를 이미지의 10%씩
                            scale=(0.8, 1.2)) # 이미지를 80% ~ 120% 사이로 Random하게 영역 확장.  scale을 (0.1, 1.5)로 해볼것
repeat_aug(count=6, org_image=org_img_pil, label='RandomAffine', transform=transform)

# 개별 변환만 적용하려면, 해당 변환만 값을 입력하고, 나머지 변환은 None. 아래는 degrees만 적용. 
transform = T.RandomAffine(degrees=30, # -30 ~ 30도 Random 회전
                            translate=None, # horizontal과 vertical방향으로 Random shift를 이미지의 10%씩
                            scale=None) # 이미지를 80% ~ 120% 사이로 Random하게 영역 확장.  scale을 (0.1, 1.5)로 해볼것
repeat_aug(count=6, org_image=org_img_pil, label='RandomAffine', transform=transform)

transform = T.RandomApply([T.RandomAffine(degrees=30, 
                            translate=(0.1, 0.1),
                            scale=(0.8, 1.2))], p=0.5) 
repeat_aug(count=6, org_image=org_img_pil, label='RandomAffine', transform=transform)

### ColorJitter
* 밝기(brightness), 대조(contrast), 채도(saturation), 색조(hue)를 함께 무작위로 변경.
* 확률값 파라미터가 없으므로 RandomApply로 확률 설정

In [ ]:
transform = T.ColorJitter(brightness=0.5, #밝기를 (1-0.5 ~ 1+0.5) 사이로, 최소값은 0임. [max(0, 1 - brightness), 1 + brightness] 
                           contrast=0.5, #대조를 (1-0.5 ~ 1+0.5) 사이로, 최소값은 0임.  
                           saturation=0.5, #채도를 (1-0.5 ~ 1+0.5) 사이로, 최소값은 0임.
                           hue=0.1 # 색조를 -0.1 ~ 0.1 사이로, 최소는 -0.5, 최대는 0.5
                          )
repeat_aug(count=6, org_image=org_img_pil, label='ColorJitter', transform=transform)

### GaussianBlur
* Blur는 전체적인 경계선을 흐리게 하거나 Noise를 제거하는 효과(Smoothing). blurring을 위한 filter 를 적용함. 
* 일반적인 Blur는 filter kernel size내의 픽셀값을 평균함. Gaussian Blur는 Gaussian 함수를 통해 kernel 값을 생성
* kernel size가 클수록 이미지가 더 흐림. Gaussian blur의 경우는 kernel size가 홀수가 되어야 함.

In [ ]:
transform = T.Compose(
    [T.Resize(size=224), T.GaussianBlur(kernel_size=(7, 7))])
repeat_aug(count=4, org_image=org_img_pil, label='Gaussian Blur', transform=transform)

### RandomChoice를 이용하여 여러개의 변환 중 하나의 변환을 선택적으로 적용
* 주로 강한 변환이 일어나는 transform들 중에 하나를 선택하기 위해서 사용됨.
* RandomChoice는 인자로 주어진 transforms 중에 반드시 하나를 선택하므로 RandomChoice자체를 확률로 설정하기 위해서는 RandomApply를 적용

In [ ]:
#CenterCrop, ColorJitter, Rotate 중에 하나를 0.5, 0.3, 0.2 의 확률로 선택(p가 list로 입력되어야 함. )
transform = T.RandomChoice([T.CenterCrop(size=224),
                             T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
                             T.RandomRotation(degrees=90)],
                           p=[0.5, 0.3, 0.2])
repeat_aug(count=6, org_image=org_img_pil, label='RandomChoice', transform=transform)

transform = T.RandomApply([
                           T.RandomChoice([T.CenterCrop(size=224),
                                            T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
                                            T.RandomRotation(degrees=90)],
                                           p=[0.5, 0.3, 0.2])
                           ], p=0.5)
repeat_aug(count=6, org_image=org_img_pil, label='RandomChoice', transform=transform)

### ToTensor와 Normalize 적용 시 유의사항
* ToTensor() 입력 인자로 PIL 또는 Numpy array를 받아서 Tensor로 반환. Tensor가 입력 되어서는 안됨. 입력을 0 ~ 1 사이값으로 변환
* Normalize()는 입력 인자로 반드시 Tensor가 입력되어야 함.

In [ ]:
transform = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomApply([T.RandomAffine(degrees=30, translate=(0.1, 0.1),scale=(0.8, 1.2))
                   ], p=0.2),
    T.ToTensor(),#Tensor 변환이 되지 않으면 Normalize는 오류 발생. 
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
aug_img = transform(org_img_pil)
print(type(aug_img))


### Flowers 데이터 세트를 이용하여 이미지에 Augmentation 적용 및 모델 학습
* 서로 다른 Augmentation을 적용하면서 모델 성능 영향이 어떻게 되는지 확인
* 처음에는 Light한 Augmentation을, 이후에는 좀 더 Heavy한 Augmentation을 적용. 

#### Flowers 데이터 세트 다운로드 및 메타 데이터 생성
* 총 3670장의 이미지로 구성되어 있으며 꽃 유형(daisy, dandelion, rose, sunflower, tulip)을 판별하기 위한 이미지 데이터 세트
* 개별 디렉토리 명이 꽃이름으로 되어 있음
* 전체의 70%를 학습, 30%를 테스트로 분할뒤, 다시 학습 데이터의 80%를 학습, 20%를 검증으로 분할

In [ ]:
!ls /kaggle/input/

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

def create_flowers_meta_df(file_dir):
    paths = [] # 이미지 파일 경로 리스트
    labels = [] # 꽃 종류
    
    # os.walk()를 이용하여 특정 디렉토리 밑에 있는 모든 하위 디렉토리를 모두 조사. 
    # kaggle/input/flowers-dataset 하위 디렉토리 밑에 jpg 확장자를 가진 파일이 모두 이미지 파일임
    # kaggle/input/flowers-dataset 밑으로 하위 디렉토리 존재
    for dirname, _, filenames in os.walk(file_dir):
        for filename in filenames:
            # 이미지 파일이 아닌 파일도 해당 디렉토리에 있음.
            if '.jpg' in filename:
                # 파일의 절대 경로를 file_path 변수에 할당. 
                file_path = dirname+'/'+ filename
                paths.append(file_path)
                
                # 파일의 절대 경로에 daily, dandelion, roses, sunflowers, tulips에 따라 labels에 값 할당.
                if 'daisy' in file_path:
                    labels.append('daisy')
                elif 'dandelion' in file_path:
                    labels.append('dandelion')
                elif 'roses' in file_path:
                    labels.append('rose')
                elif 'sunflowers' in file_path:
                    labels.append('sunflowers')
                elif 'tulips' in file_path:
                    labels.append('tulips')
    # DataFrame 메타 데이터 생성. 
    data_df = pd.DataFrame({'path':paths, 
                            'label':labels})
    # Target값  변환
    label_mapping = {'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflowers': 3, 'tulips': 4}
    data_df['target'] = data_df['label'].map(label_mapping)

    return data_df

In [ ]:
from sklearn.model_selection import train_test_split

data_df = create_flowers_meta_df('/kaggle/input/') # /kaggle/input/flowers-dataset

# 전체 데이터 세트에서 학습(전체의 70%)과 테스트용(전체의 30%) 메타 정보 DataFrame 생성.
train_df, test_df = train_test_split(data_df, test_size=0.3, stratify=data_df['target'], random_state=2025)
# 기존 학습 DataFrame을 다시 학습과 검증 DataFrame으로 분할. 80%가 학습, 20%가 검증
tr_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['target'], random_state=2025)

print(data_df.shape, train_df.shape, tr_df.shape, val_df.shape, test_df.shape)

In [ ]:
print(data_df['label'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# 이미지를 시각화
def show_grid_images(image_paths, labels, ncols=6):
    figure, axs = plt.subplots(figsize=(22, 6), nrows=1, ncols=ncols)
    for i in range(ncols):
        image = Image.open(image_paths[i])
        axs[i].imshow(image)
        axs[i].set_title(labels[i])

daisy_image_paths = data_df[data_df['label']=='daisy']['path'].iloc[:6].tolist()
daisy_labels = data_df[data_df['label']=='daisy']['label'].iloc[:6].tolist()
show_grid_images(daisy_image_paths, daisy_labels, ncols=6)

rose_image_paths = data_df[data_df['label']=='rose']['path'].iloc[:6].tolist()
rose_labels = data_df[data_df['label']=='rose']['label'].iloc[:6].tolist()
show_grid_images(rose_image_paths, rose_labels, ncols=6)

### Flower Custom Dataset 및 DataLoader 생성

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image

class FlowerDataset(Dataset):
    # 이미지 파일리스트, 타겟 파일리스트, transforms 등 이미지와 타겟 데이터 가공에 필요한 인자들을 입력 받음
    def __init__(self, image_paths, targets=None, transform=None):
        self.image_paths = image_paths
        self.targets = targets
        self.transform = transform
    
    # 전체 건수를 반환
    def __len__(self):
        return len(self.image_paths)
        
    # idx로 지정된 하나의 image, label을 tensor 형태로 반환
    def __getitem__(self, idx):
        # PIL을 이용하여 이미지 로딩하고 PIL Image 객체 반환.
        pil_image = Image.open(self.image_paths[idx])
        # 보통은 transform이 None이 되는 경우는 거의 없음(Tensor 변환이라도 있음)
        image = self.transform(pil_image)
        
        if self.targets is not None:
            # 개별 target값을 tensor로 변환.
            target = torch.tensor(self.targets[idx])
            return image, target
        # 테스트 데이터의 경우 targets가 입력 되지 않을 수 있으므로 이를 대비. 
        else:
            return image

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms as T

BATCH_SIZE = 32

#학습과 검증용 DataLoader 생성 함수
def create_tr_val_loader(tr_df, val_df, tr_transform, val_transform):
    tr_dataset = FlowerDataset(image_paths=tr_df['path'].to_list(),
                               targets=tr_df['target'].to_list(), transform=tr_transform)
    val_dataset = FlowerDataset(image_paths=val_df['path'].to_list(),
                               targets=val_df['target'].to_list(), transform=val_transform)
    tr_loader= DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=2*BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
    
    return tr_loader, val_loader

In [ ]:
IMG_SIZE = 224
IMG_MEANS = [0.485, 0.456, 0.406] # ImageNet 데이터세트의 이미지 채널별 평균값
IMG_STD = [0.229, 0.224, 0.225] # ImageNet 데이터세트의 이미지 채널별 표준편차값

tr_transform = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

val_transform = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df,
                                             tr_transform=tr_transform, val_transform=val_transform)
images, labels = next(iter(tr_loader))
print(images.shape, labels.shape)

### 모델 생성 후 학습 및 평가 수행
* torchvision pretrained resnet 50 모델 생성
* 서로 다른 유형의 Augmentation을 적용하면서 성능 비교

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

def create_resnet_model(model_name, num_classes=10, weights='DEFAULT'):
    model = None
    if model_name == 'resnet50':
        model = models.resnet50(weights=weights)
    elif model_name == 'resnet101':
        model = models.resnet101(weights=weights)
    
    num_in_features = model.fc.in_features
    model.fc = nn.Linear(in_features=num_in_features, out_features=num_classes)

    return model
    
resnet_model = create_resnet_model('resnet50', num_classes=5, weights='DEFAULT') #resnet50, resnet101

### Trainer 클래스 생성 및 적용
* Trainer 클래스는 https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true 로 download 후 import 함

In [ ]:
# /kaggle/working/modular/v1 디렉토리에 utils.py 파일 다운로드
!rm -rf ./modular/v1
!mkdir -p ./modular/v1
!wget -O ./modular/v1/utils.py https://raw.githubusercontent.com/chulminkw/CNN_PG_Torch/main/modular/v1/utils.py?raw=true
!ls ./modular/v1

In [ ]:
import sys

# 반드시 system path를 아래와 같이 잡아줘야 함. 
sys.path.append('/kaggle/working')

#아래가 수행되는지 반드시 확인
from modular.v1.utils import Trainer, Predictor, ModelCheckpoint, EarlyStopping

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# 5개의 꽃 종류
NUM_CLASSES = 5

def train_flowers_with_aug(tr_df, val_df, tr_transform, val_transform, epochs=20):
    tr_loader, val_loader = create_tr_val_loader(tr_df=tr_df, val_df=val_df, 
                                                 tr_transform=tr_transform, val_transform=val_transform)
    model = create_resnet_model('resnet50', num_classes=NUM_CLASSES, weights='DEFAULT')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    optimizer = Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    # 여러 augmentation 비교를 위해서 고정된 패턴의 학습율 적용을 위해 StepLR 선택.
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
    # callbacks는 None
    trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
                      train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, callbacks=None, 
                      device=device)
    # 학습 및 평가.
    history = trainer.fit(epochs)
    #학습된 trainer와 history 반환. 
    return trainer, history

#### Augmentation을 적용하지 않은 transform을 이용하여 학습 및 평가

In [ ]:
IMG_SIZE = 224
IMG_MEANS = [0.485, 0.456, 0.406] # ImageNet 데이터세트의 이미지 채널별 평균값
IMG_STD = [0.229, 0.224, 0.225] # ImageNet 데이터세트의 이미지 채널별 표준편차값

tr_transform = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

val_transform = T.Compose([
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

trainer, history = train_flowers_with_aug(tr_df, val_df, 
                                          tr_transform=tr_transform, val_transform=val_transform,
                                          epochs=30)

In [ ]:
from modular.v1.utils import Predictor

test_image_paths = test_df['path'].to_list()
test_targets = test_df['target'].to_list()

IMG_SIZE=224
test_transform = T.Compose([
                        T.Resize(size=(IMG_SIZE, IMG_SIZE)),
                        T.ToTensor(), 
                        T.Normalize(mean=[0.485, 0.456, 0.406], 
                                    std=[0.229, 0.224, 0.225])
])

test_dataset = FlowerDataset(image_paths=test_image_paths, 
                            targets=test_targets, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

trained_model = trainer.get_trained_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### Augmentation을 적용하여 학습 및 평가수행 - 01
* Flip 계열만 적용

In [ ]:
tr_transform_aug_01 = T.Compose([
            T.RandomHorizontalFlip(0.5),
            T.RandomVerticalFlip(0.5),
            T.Resize(size=(IMG_SIZE, IMG_SIZE)),
            T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
])

trainer, history = train_flowers_with_aug(tr_df=tr_df, val_df=val_df, 
                                          tr_transform=tr_transform_aug_01, val_transform=val_transform,
                                          epochs=30)

In [ ]:
trained_model = trainer.get_trained_model()

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### Augmentation을 적용하여 학습 및 평가수행 - 02
* Flip, Affine, ColorJitter 함께 적용

In [ ]:
tr_transform_aug_02 = T.Compose([
    T.RandomHorizontalFlip(p=0.3),
    T.RandomVerticalFlip(p=0.3),
    T.RandomApply([T.RandomAffine(degrees=30, translate=(0.1, 0.1),scale=(0.8, 1.2))
                   ], p=0.2),
    T.RandomApply([T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.3),
    T.Resize(size=(IMG_SIZE, IMG_SIZE)),
    T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
    
])

trainer, history = train_flowers_with_aug(tr_df=tr_df, val_df=val_df, 
                                          tr_transform=tr_transform_aug_02, val_transform=val_transform,
                                          epochs=30)

In [ ]:
trained_model = trainer.get_trained_model()

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### Augmentation을 적용하여 학습 및 평가수행 - 03
* CenterCrop(RandomResizedCrop)을 적용. Crop 계열은 강한 Augmentation. 좋은 결과를 얻을 수 있지만, 이미지 특성에 따라 성능이 떨어질 수도 있음

In [ ]:
tr_transform_aug_03 = T.Compose([
    T.RandomHorizontalFlip(p=0.3),
    T.RandomVerticalFlip(p=0.3),
    T.RandomApply([T.CenterCrop(size=(200,200))], p=0.4),
    T.Resize(size=(IMG_SIZE, IMG_SIZE)),
    T.ToTensor(), T.Normalize(mean=IMG_MEANS, std=IMG_STD)
    
])

trainer, history = train_flowers_with_aug(tr_df=tr_df, val_df=val_df, 
                                          tr_transform=tr_transform_aug_03, val_transform=val_transform,
                                          epochs=30)

In [ ]:
trained_model = trainer.get_trained_model()

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')